# Linear Algebra of Red Black Graphs

## Introduction
We have provided a formal definition for a Red Black Graph, looked at the adjacency matrix and it's transitive closure. We've discussed *pedigree numbers* and the *avos* product for scalers and looked at a number of interesting properites derived. We'll now extend these observations into a more general discussion of how principles of linear algebra can be applied to Red Black Graphs.

## Vector Classes
Within the context of a Red Black Graph and it's adjacency matrix, we define the following vector classes:

* *row* vector - represented by *u*. These vectors represent ancestry for a given vertex. Values for elements in these vectors are constrained to natural numbers. Any positive integer may appear in an element at most once. One may appear only if the vertex is black.
* *column* vector - represented by *v*. These vectors represent descendency for a given vertex. Values for elements in these vectors are also constrained to natural numbers. Furthermore if the vertex is red, they are constrained to even numbers, if the vertex is black, they are constrained to odd numbers (plus zero). One may appear only once, otherwise there are no constraints. 
* *simple row vector* - represented by *u<sub>s</sub>*. Row vectors for which elements are constrained to {0, 1, 2, 3}. These represent a given vertex and it's immediate ancestry only.
* *simple column vector* - represented by *v<sub>s</sub>*. Column vectors for which elements are constrained to {0, 1, 2, 3}. These represent a given vertex and it's immediate descendency only.
* *closed row vector* - represented by *u<sub>c</sub>*. Row vectors from $A_{rb}^+$. These represent the complete ancestry for a given vertex.
* *closed column vector* - represented by *v<sub>c</sub>*. Column vectors from $A_{rb}^+$. These represent the complete descendency for a given vertex.

## *avos* Product for Vectors

Let's consider what an *avos* product might represent. Given a row vector and a column vector, we'll define the *avos* product to yield the *pedigree number* representing the relationship between the vertices representing the row and column vectors respectively. 

Consider the Red Black adjacency matrix from our prior example:

\begin{vmatrix}
0 & 2 & 3 & 0 & 0 \\
0 & 0 & 0 & 2 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
2 & 0 & 0 & 0 & 1 \\
\end{vmatrix}

Assuming 0 indexing in the tha matrix, the 4th row vector is a simple row vector for $vertex_{4}$ while the 2nd column vector is simple column for $vertex_{2}$. It is trivally observable that that $vertex_{4}$ is related to $vertex_{2}$ (by *pedigree number* 5). 

\begin{equation*}
\begin{vmatrix}
2 & 0 & 0 & 0 & 1 \\
\end{vmatrix}
\cdot
\begin{vmatrix}
3 \\
0 \\
1 \\
0 \\
0 \\
\end{vmatrix}
= 5
\end{equation*}

The vector dot product, summing element-wise products, results in a scaler value of 6. Element wise *avos* product does in fact yield *pedigree number* that represents a relationship. However, it is possible for there to be multiple paths through the graph bewteen two nodes. If that were the case, summing the element-wise *avos* product would not result in the *pedigree number* representing the relationship from $vertex_{a}$ to $vertex_{c}$. For the *avos* vector product, rather than summing the element-wise *avos* products we arbitrarily choos the non-zero minimum element-wise product, thus representing the "closest" relationship between $vertex_{a}$ and $vertex_{c}$.

A simple implementation of the *avos* vector product follows:

In [8]:
# %load ../redblackgraph/simple/vec_avos.py
from redblackgraph.simple import avos

def vec_avos(x, y):
    '''Given two vectors, compute the "avos" dot product.'''
    return min([avos(a, b) for a, b in zip(x, y) if a > 0 and b > 0], default=0)


### Observations

The product of a simple row vector and the transitive closure of a Red Black adjacency matrix is a closed row vector
\begin{equation*}
u_{s} A_{rb}^+ = u_{c}
\end{equation*}

The product of the transitive closure of a Red Black adjacency matrix and a simple column vector is a closed column vector
\begin{equation*}
A_{rb}^+ v_{s} = v_{c}
\end{equation*}


## Matrix multiplication

With scaler and vector *avos* products defined, extension to matrices is elementary. Given *A* and *B*, both matrices following the contraints defined for Red-Black adjacency matrices, and *C* = *A* *B*, the elements of $C_{ij}$ are given by:
* if i != j, the vector *avos* product of *row vector<sub>i</sub>* from A and *column vector<sub>j</sub>* from B
* if i == j and $A_{ij}$ == $B_{ij}$, $A_{ij}$
* otherwise, undefined.

Matrix multiplication of Red-Black adjacency matrices may seem a little abstract so let's consider a practical example to motivate the analysis. Given a Red-Black adjacency matrix, *A<sub>rb</sub>*, the result of $A_{rb}^2$ shows all vertices directly related by following up to 2 relationship edges, the result of $A_{rb}^3$ shows all vertices related by following up to 3 relationship edges, etc. For some *n* <= |*V*| there will be a $A_{rb}^n$ == $A_{rb}^+$.

A simple implementation of the avox matrix product follows:

In [1]:
# %load ../redblackgraph/simple/mat_avos.py
from redblackgraph.simple import avos


def mat_avos(A, B):
    '''Given two matrices, compute the "avos" product.'''
    return [[min([avos(a, b) for a, b in zip(A_row, B_col) if a > 0 and b > 0], default=0) for B_col in zip(*B)] for
            A_row in A]


In [2]:
A = [[0, 2, 3, 0, 0], [0, 0, 0, 2, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [2, 0, 0, 0, 1]]
print(mat_avos(A, A))

[[0, 0, 3, 4, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 4, 5, 0, 1]]


## Relational Composition

The question arise how to correctly update *A<sub>rb</sub><sup>+</sup>* with the addition of new vertices to the graph. We propose an algorithm that is relatively simple in concept that generates *A<sup>'</sup><sub>rb</sub><sup>+</sup>*. Consider adding a vertex to the graph corresponding to *A<sub>rb</sub><sup>+</sup>* of size *N*. We define a relational composition that can be used to add a new node to the graph and produce *A<sup>'</sup><sub>rb</sub><sup>+</sup>* of size *N* + 1. Conceptually:

*A<sup>'</sup><sub>rb</sub><sup>+</sup>* = *v* *A<sub>rb</sub><sup>+</sup>* *u*

Precisely this relational composition is accomplished by:

1. generate *v<sub>c</sub>* (given by observation 1 above)
2. generate *u<sub>c</sub>* (given by observation 2 above)
3. *A<sup>'</sup><sub>rb</sub><sup>+</sup>* is composed by:
    1. appending *v<sub>c</sub>* as a new row
    2. appending *u<sub>c</sub>* as a new column
    3. setting element *A<sup>'</sup><sub>rb</sub><sup>+</sup>*<sub>N+1, N+1</sub> to 0 if vertex is red, and 1 if vertex is black
    4. For each row, r, where *v<sub>c<sub>i</sub></sub>* != 0, set *A<sup>'</sup><sub>rb</sub><sup>+</sup>*<sub>r,i</sub> = *v<sub>c<sub>r</sub></sub>* avos *u<sub>c<sub>i</sub></sub>*

## Areas of Further investigation

* Determinants
* Eigenvalues
* Loop prevention
* *avos* properties (commutitivity, associativity, distribution, identity, inverse, etc.)
* Spectral Graph Theory
* Explore ideas to reduce dimensionality (embedding similar to word2vec, etc.)
* Axioms of linear algebra:
    * u + (v + w) = (u + v) + w
    * v + w = w + v
    * There is a vector *0* such that *0* + v = v for all v
    * For every vector v there is a vector -v so that v + (-v) = *0*
    * a(bv) = (ab)v
    * 1v = v
    * a(v + w) = av + aw
    * (a + b)v = av + bv